In [30]:
import re
from enum import Enum
import string
import pandas as pd
import numpy as np
from anytree import Node, RenderTree

In [2]:
class State(Enum):
    INITIAL = 0, 'INITIAL'
    IDENTIFIER_START = 1, 'IDENTIFIER_START'
    IDENITIFIER_END = 2, 'IDENTIFIER_END'
    INTEGER_START = 3, 'INTEGER_START'
    INTEGER_END = 4, 'INTEGER_END'
    REAL_START = 5, 'REAL_START'
    REAL_IN = 6, 'REAL_IN'
    REAL_END = 7, 'REAL_END'
    SEPARATOR = 8, 'SEPARATOR'
    OPERATOR = 9, 'OPERATOR'
    SEPCIAL_CHARACTER = 10, 'SPECIAL_CHARACTER'
    DEAD = 11, 'DEAD_STATE'
    
    def __new__(cls, value, name):
        x = object.__new__(cls)
        x._value_ = value
        x._name = name
        return x
    
    @property
    def getvalue(self):
        return self.value

class Input(Enum):
    LETTER = 0, 'LETTER'
    DIGIT = 1, 'DIGIT'
    DOT = 2, 'DOT'
    SEP = 3, 'SEP'
    OP = 4, 'OP'
    SPECIAL = 5, 'SPECIAL'
    
    def __new__(cls, value, name):
        x = object.__new__(cls)
        x._value_ = value
        x._name = name
        return x
    
    @property
    def getvalue(self):
        return self.value
    
class Token():
    def __init__(self, **kwargs):
        self._value = kwargs['value']
        if self._value == 0:
            self._name = 'IDENTIFIER'
        elif self._value == 1:
            self._name = 'KEYWORD'
        elif self._value == 2:
            self._name = 'INTEGER'
        elif self._value == 3:
            self._name = 'REAL'
        elif self._value == 4:
            self._name = 'SEPARATOR'
        elif self._value == 5:
            self._name = 'OPERATOR'
        elif self._value == 6:
            self._name = 'SPECIAL'
        else:
            raise ValueError('value should be within 0 and 5')
        self._lexeme = kwargs['lexeme']
    
    @property
    def getvalue(self):
        return self._value
    
    @property
    def lexeme(self):
        return self._lexeme
    
    @property
    def name(self):
        return self._name

In [3]:
# GLOBAL VARIABLES
SEPARATORS = ['\(','\)','[',']','\{','\}','.',',',':',';',' ','\cdot']
OPERATORS = ['+','-','=','/','>','<','%',' ','\&','\times','\div','\ast','\cup','\cap','\subset','\supset','\subseteq','\|','\perp','\eq','\geq']
STATE_TABLE_DATA = [
    [1,3,11,8,9,10],
    [1,2,11,2,2,2],
    [0,0,0,0,0,0],
    [4,3,5,4,4,4],
    [0,0,0,0,0,0],
    [11,6,11,11,11,11],
    [7,6,7,7,7,7],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0]
]
STATE_TABLE = [[None for i in range(len(STATE_TABLE_DATA[0]))] for j in range(len(STATE_TABLE_DATA))]
for i in range(len(STATE_TABLE_DATA)):
    for j in range(len(STATE_TABLE_DATA[0])):
        STATE_TABLE[i][j] = State(STATE_TABLE_DATA[i][j])

KEYWORDS = ['sin','cos','tan','sinh','cosh','tanh','and','or']
GREEKS = [r'\sigma',r'\Sigma',r'\gamma',r'\delta',r'\Delta',r'\eta',r'\theta',r'\epsilon',r'\lambda',r'\mu',r'\Pi',r'\rho',r'\phi',r'\omega',r'\ohm']
SPECIAL_CHARS = ['\infty','\exists','\forall','\#','\$'] + GREEKS

In [4]:
get_characters = lambda chars: [c if c not in ['\n', '\t', '\r'] else ' ' for c in chars]

In [5]:
checkLetter = lambda chars: True if re.match("[a-zA-Z]", chars) else False
checkDigit = lambda chars: True if re.match("[0-9]", chars) else False
checkDollar = lambda chars: True if chars in ['$'] else False
checkDot = lambda chars: True if chars in ['.','\cdot'] else False
checkSep = lambda chars: True if chars in SEPARATORS else False
checkOp = lambda chars: True if chars in OPERATORS else False
checkSpecial = lambda chars: True if chars in SPECIAL_CHARS else False
checkEscape = lambda chars: True if chars in ["\\"] else False

In [6]:
# checks each input and returns an instance of input class
def checkInput(chars):
    if checkLetter(chars):
        return Input.LETTER
    elif checkDigit(chars):
        return Input.DIGIT
    elif checkDollar(chars):
        return Input.DOLLAR
    elif checkDot(chars):
        return Input.DOT
    elif checkSep(chars):
        return Input.SEP
    elif checkOp(chars):
        return Input.OP
    elif checkSpecial(chars):
        return Input.SPECIAL
    else:
        raise ValueError("INVALID INPUT")

In [7]:
def lexer(expression):
    expression = expression + [' '] # added this because integers, letters and real require a separator at the end to move to final state
    prev_state = State.INITIAL
    lexeme = ''
    tokens = []
    i = 0
    while i < len(expression):
        backup = False
        final = False
        alpha = expression[i]
        x = prev_state.getvalue
        y = checkInput(alpha).getvalue
        current_state = STATE_TABLE[x][y]
        if current_state.getvalue == 2:
            if lexeme in KEYWORDS:
                tokens.append(Token(value=1, lexeme=lexeme))
            else:
                tokens.append(Token(value=0, lexeme=lexeme))
            final = True
            backup = True
        elif current_state.getvalue == 4:
            tokens.append(Token(value=2, lexeme=lexeme))
            final = True
            backup = True
        elif current_state.getvalue == 7:
            tokens.append(Token(value=3, lexeme=lexeme))
            final = True
            backup = True
        elif current_state.getvalue == 8:
            tokens.append(Token(value=4, lexeme=alpha))
            final = True
            backup = False
        elif current_state.getvalue == 9:
            tokens.append(Token(value=5, lexeme=alpha))
            final = True
            backup = False
        elif current_state.getvalue == 10:
            tokens.append(Token(value=6, lexeme=alpha))
            final = True
            backup = False
        elif current_state.getvalue == 11:
            raise ValueError("Reached Dead State")
        if final:
            lexeme = ''
            prev_state = State.INITIAL
        else:
            lexeme += alpha
            prev_state = current_state
        if not backup:
            i += 1
    return tokens[:-1]

In [8]:
expression1 = [r'z', r'=', r'x',r'+',r'y']
expression2 = ['s','i','n',r'\theta','+','c','o','s',r'\theta','=','1']

In [9]:
tokens = lexer(expression1)

In [10]:
for i in tokens:
    print(i.lexeme)

z
=
x
+
y


## Parser

In [11]:
class NonTerminals(Enum):
    S = 0, 'S'
    E = 1, 'E'
    Q = 2, 'Q'
    T = 3, 'T'
    R = 4, 'R'
    F = 5, 'F'
    NONE = 6, 'NONE'
    
    def __new__(cls, value, name):
        x = object.__new__(cls)
        x._value_ = value
        x._name = name
        return x
        
    @property
    def getvalue(self):
        return self.value
    
    @property
    def name(self):
        return self._name

class Terminals(Enum):
    IDENTIFIER = 0, 'IDENTIFIER'
    EQUAL = 1, 'EQUAL'
    PLUS = 2, 'PLUS'
    MINUS = 3, 'MINUS'
    MULTIPLY = 4, 'MULTIPLY'
    DIVISION = 5, 'DIVISION'
    MOD = 6, "MOD"
    LEFT_ROUNDB = 7, 'LEFT_ROUNDB'
    RIGHT_ROUNDB = 8, 'RIGHT_ROUNDB'
    LEFT_SQUAREB = 9, 'LEFT_SQUAREB'
    RIGHT_SQUAREB = 10, 'RIGHT_SQUAREB'
    LEFT_CURLYB = 11, 'LEFT_CURLYB'
    RIGHT_CURLYB = 12, 'RIGHT_CURLYB'
    AND = 13, 'AND'
    OR = 14, 'OR'
    INVI_TIMES = 15, 'INVI_TIMES'
    DOLLAR = 16, "DOLLAR"
    NONE = 17, 'NONE'
    
    def __new__(cls, value, name):
        x = object.__new__(cls)
        x._value_ = value
        x._name = name
        return x
    
    @property
    def getvalue(self):
        return self.value
    
    @property
    def name(self):
        return self._name

class ProdRules(Enum):
    IEE = 0, "i = E"
    E = 1, "E"
    TQ = 2, "T Q"
    ATQ = 3, "+ T Q"
    STQ = 4, "- T Q"
    FR = 5, "F R"
    MFR = 6, "* F R"
    DFR = 7, "/ F R"
    PFR = 8, "% F R"
    BEB = 9, "( E )"
    I = 10, "i"
    EPS = 11, "EPSILON"
    INVALID = 12, "INVALID"
    
    def __new__(cls, value, name):
        x = object.__new__(cls)
        x._value_ = value
        x._name = name
        return x
    
    @property
    def getvalue(self):
        return self.value

In [12]:
# Parser Global Variables
PREDICTIVE_PARSER_TABLE_DATA = [
    [0, 12, 12, 12, 12, 12, 12, 1, 12, 12, 12, 12, 12, 12, 12, 12, 12],
    [2, 12, 12, 12, 12, 12, 12, 2, 12, 12, 12, 12, 12, 12, 12, 12, 12],
    [12, 12, 3, 4, 12, 12, 12, 12, 11, 12, 12, 12, 12, 12, 12, 12, 11],
    [5, 12, 12, 12, 12, 12, 12, 5, 12, 12, 12, 12, 12, 12, 12, 12, 12],
    [12, 12, 11, 11, 6, 7, 8, 12, 11, 12, 12, 12, 12, 12, 12, 12, 11],
    [10, 12, 12, 12, 12, 12, 12, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12],
]

PREDICTIVE_PARSER_TABLE = [[None for i in range(len(PREDICTIVE_PARSER_TABLE_DATA[0]))] for j in range(len(PREDICTIVE_PARSER_TABLE_DATA))]
for i in range(len(PREDICTIVE_PARSER_TABLE_DATA)):
    for j in range(len(PREDICTIVE_PARSER_TABLE_DATA[0])):
        PREDICTIVE_PARSER_TABLE[i][j] = ProdRules(PREDICTIVE_PARSER_TABLE_DATA[i][j])

PREDICTIVE_PARSER_TABLE = np.array(PREDICTIVE_PARSER_TABLE)

LIST_TERMINALS = [i for i in Terminals if i.name not in ["INVALID", "EPSILON"] ]

In [13]:
PREDICTIVE_PARSER_TABLE.shape

(6, 17)

In [14]:
def change_terminal(lexeme):
    if lexeme == "i":
        return Terminals.IDENTIFIER
    elif lexeme == "=":
        return Terminals.EQUAL
    elif lexeme == "+":
        return Terminals.PLUS
    elif lexeme == "-":
        return Terminals.MINUS
    elif lexeme in ["*", r"\times", r"\ast"]:
        return Terminals.MULTIPLY
    elif lexeme in ["/", r"\div"]:
        return Terminals.DIVISION
    elif lexeme == "%":
        return Terminals.MOD
    elif lexeme in ["(", r"\("]:
        return Terminals.LEFT_ROUNDB
    elif lexeme in [")", r"\)"]:
        return Terminals.RIGHT_ROUNDB
    elif lexeme == "[":
        return Terminals.LEFT_SQUAREB
    elif lexeme == "]":
        return Terminals.RIGHT_SQUAREB
    elif lexeme in ["{", r"\{"]:
        return Terminals.LEFT_CURLYB
    elif lexeme in ["}", r"\}"]:
        return Terminals.RIGHT_CURLYB
    elif lexeme in ["and", r"\&"]:
        return Terminals.AND
    elif lexeme in ["or", r"\|"]:
        return Terminals.OR
    elif lexeme == "$":
        return Terminals.DOLLAR
    else:
        return Terminals.NONE

def changeinput(token, lexeme):
    if token == "IDENTIFIER":
        return Terminals.IDENTIFIER
    elif token == "KEYWORD":
        if lexeme in ["and", r"\&"]:
            return Terminals.AND
        elif lexeme in ["or", r"\|"]:
            return Terminals.OR
        else:
            return Terminals.NONE
    elif token == "OPERATOR":
        if lexeme == "+":
            return Terminals.PLUS
        elif lexeme == "-":
            return Terminals.MINUS
        elif lexeme in ["*", r"\times", r"\ast"]:
            return Terminals.MULTIPLY
        elif lexeme in [r"/", r"\div"]:
            return Terminals.DIVISION
        elif lexeme == "%":
            return Terminals.MOD
        elif lexeme == "$":
            return Terminals.DOLLAR
        elif lexeme == "=":
            return Terminals.EQUAL
        else:
            return Terminals.NONE
    elif token == "SEPARATOR":
        if lexeme in ["(", r"\("]:
            return Terminals.LEFT_ROUNDB
        elif lexeme in [")", r"\)"]:
            return Terminals.RIGHT_ROUNDB
        elif lexeme == "[":
            return Terminals.LEFT_SQUAREB
        elif lexeme == "]":
            return Terminals.RIGHT_SQUAREB
        elif lexeme in ["{", r"\{"]:
            return Terminals.LEFT_CURLYB
        elif lexeme in ["}", r"\}"]:
            return Terminals.RIGHT_CURLYB
        elif lexeme == "$":
            return Terminals.DOLLAR
        elif lexeme == ";":
            return Terminals.DOLLAR
        else:
            return Terminals.NONE
    else:
        return Terminals.NONE

In [15]:
stack = []
#Helper function used to pop the stack
def pop(stack):
    temp = stack[-1]
    del stack[-1]
    return temp

#Helper function used to push contents to the stack
def push(stack, T):
    stack.append(T)

In [16]:
#Helper function used to push the contents to the stack according to the Production Rules
def pushRules(rule):
    if rule == ProdRules(0):
        push(stack, NonTerminals.E)
        push(stack, Terminals.EQUAL)
        push(stack, Terminals.IDENTIFIER)
    elif rule == ProdRules(1):
        push(stack, NoneTerminals.E)
    elif rule == ProdRules(2):
        push(stack, NonTerminals.Q)
        push(stack, NonTerminals.T)
    elif rule == ProdRules(3):
        push(stack, NonTerminals.Q)
        push(stack, NonTerminals.T)
        push(stack,Terminals.PLUS)
    elif rule == ProdRules(4):
        push(stack,NonTerminals.Q)
        push(stack,NonTerminals.T)
        push(stack, Terminals.MINUS)
    elif rule == ProdRules(5):
        push(stack, NonTerminals.R)
        push(stack, NonTerminals.F)
    elif rule == ProdRules(6):
        push(stack, NonTerminals.R)
        push(stack, NonTerminals.F)
        push(stack, Terminals.MULTIPLY)
    elif rule == ProdRules(7):
        push(stack, NonTerminals.R)
        push(stack, NonTerminals.F)
        push(stack, Terminals.DIVISION)
    elif rule == ProdRules(8):
        push(stack, NonTerminals.R)
        push(stack, NonTerminals.F)
        push(stack, Terminals.MOD)
    elif rule == ProdRules(9):
        push(stack, Terminals.RIGHT_ROUNDB)
        push(stack, NonTerminals.E)
        push(stack, Terminals.LEFT_ROUNDB)
    elif rule == ProdRules(10):
        push(stack, Terminals.IDENTIFIER)
    elif rule == ProdRules(11):
        pass                      #This statement does nothing, used so the code does not given runtime error when function is called for epsilom
    else:
        push(stack, NonTerminals.NONE) 

In [27]:
def parser(tokens_lexemes):
    tokens_lexemes.append(Token(value=4, lexeme="$"))
    per_ip = [tokens_lexemes[0]]
    result = []
    i = 0                                                       #used to keep track of the length of the tokens_lexemes dataframe
    push(stack, NonTerminals.S)                                 #push S to stack for the first line
    while i < len(tokens_lexemes) and len(stack) != 0:   #executes until the stack becomes empty and the tokens_lexmes dataframe reaches the end
        # print(stack)
        x = changeinput(tokens_lexemes[i].name, tokens_lexemes[i].lexeme)       #changes input to the instance of Terminals class; will be used further as column no for parser table
        y = pop(stack)                                                            #pops the first value of the stack; will be used further as row no for parser table
        if (x == Terminals.NONE or y == NonTerminals.NONE):                       #this will raise a Syntax Error which will be caught by Error Handling
            raise SyntaxError("Token: {}\nLexeme: {} \nSyntax Error: Invalid Syntax".format(tokens_lexemes[i].name, tokens_lexemes[i].lexeme))
        elif (y in LIST_TERMINALS):                             #executes if y and x are both equal
            if (x == y):
                result.append(per_ip)
                i+=1
            else:
                raise SyntaxError("Token: {}\nLexeme: {} \nSyntax Error: Invalid Syntax".format(tokens_lexemes[i].name, tokens_lexemes[i].lexeme))
            per_ip = [tokens_lexemes[i]]           #used to keep record of the token and lexeme for the current iteration; will be added to op_df when the input changes
        else:                                                                   #executes when y and x are not equal
            new_value = PREDICTIVE_PARSER_TABLE[y.value][x.value]                       #calculates the Production Rule according to y and x from the parser table
            per_ip.append((y, new_value))                                          #adds the Prod Rule used to the op_ls list
            pushRules(new_value)                                                  #pushes the contents of the Prod Rule to the stack
    result.append(per_ip)
    return result

In [28]:
tree = parser(tokens)
print(len(stack))

0


In [29]:
tree

[[<__main__.Token at 0x1c600afd4a8>,
  (<NonTerminals.S: 0>, <ProdRules.IEE: 0>)],
  (<NonTerminals.E: 1>, <ProdRules.TQ: 2>),
  (<NonTerminals.T: 3>, <ProdRules.FR: 5>),
  (<NonTerminals.F: 5>, <ProdRules.I: 10>)],
  (<NonTerminals.R: 4>, <ProdRules.EPS: 11>),
  (<NonTerminals.Q: 2>, <ProdRules.ATQ: 3>)],
  (<NonTerminals.T: 3>, <ProdRules.FR: 5>),
  (<NonTerminals.F: 5>, <ProdRules.I: 10>)],
  (<NonTerminals.R: 4>, <ProdRules.EPS: 11>),
  (<NonTerminals.Q: 2>, <ProdRules.EPS: 11>)]]

In [37]:
def add_prod_rules(rule, node_parent):
    if rule == ProdRules(0):
        E = Node(NonTerminals.E.name, parent=node_parent)
        EQ = Node(Terminals.EQUAL.name, parent=node_parent)
        ID = Node(Terminals.IDENTIFIER.name, parent=node_parent)
    elif rule == ProdRules(1):
        E = Node(NonTerminals.E.name, parent=node_parent)    
    elif rule == ProdRules(2):
        Q = Node(NonTerminals.Q.name, parent=node_parent)
        T = Node(NonTerminals.T.name, parent=node_parent)
    elif rule == ProdRules(3):
        Q = Node(NonTerminals.Q.name, parent=node_parent)
        T = Node(NonTerminals.T.name, parent=node_parent)
        PL = Node(Terminals.PLUS, parent=node_parent)
    elif rule == ProdRules(4):
        Q = Node(NonTerminals.Q.name, parent=node_parent)
        T = Node(NonTerminals.T.name, parent=node_parent)
        MI = Node(Terminals.MINUS, parent=node_parent)
    elif rule == ProdRules(5):
        R = Node(NonTerminals.R.name, parent=node_parent)
        F = Node(NonTerminals.F.name, parent=node_parent)
    elif rule == ProdRules(6):
        R = Node(NonTerminals.R.name, parent=node_parent)
        F = Node(NonTerminals.F.name, parent=node_parent)
        MU = Node(Termimals.MULTIPLY, parent=node_parent)
    elif rule == ProdRules(7):
        R = Node(NonTerminals.R.name, parent=node_parent)
        F = Node(NonTerminals.F.name, parent=node_parent)
        DIV = Node(Termimals.DIVISION, parent=node_parent)
    elif rule == ProdRules(8):
        R = Node(NonTerminals.R.name, parent=node_parent)
        F = Node(NonTerminals.F.name, parent=node_parent)
        MOD = Node(Termimals.MOD, parent=node_parent)
    elif rule == ProdRules(9):
        RIGHTB = Node(Terminals.RIGHT_ROUNDB.name, parent=node_parent)
        E = Node(NonTerminals.E.name, parent=node_parent)
        LEFTB = Node(Terminals.LEFT_ROUNDB.name, parent=node_parent)
    elif rule == ProdRules(10):
        ID = Node(Terminals.IDENTIFIER.name, parent=node_parent)
    elif rule == ProdRules(11):
        pass                      #This statement does nothing, used so the code does not given runtime error when function is called for epsilom
    else:
        pass

In [38]:
# define tree
S = Node("S")

In [40]:
nodes = []
for i in tree:
    if type(i[-1]) in ["list", "tuple"]:
        for j in i[1:]:
            node_parent = 

[<__main__.Token object at 0x000001C600AFD4A8>, (<NonTerminals.S: 0>, <ProdRules.IEE: 0>)]
[<__main__.Token object at 0x000001C600AFD630>, (<NonTerminals.E: 1>, <ProdRules.TQ: 2>), (<NonTerminals.T: 3>, <ProdRules.FR: 5>), (<NonTerminals.F: 5>, <ProdRules.I: 10>)]
[<__main__.Token object at 0x000001C600AFD668>, (<NonTerminals.R: 4>, <ProdRules.EPS: 11>), (<NonTerminals.Q: 2>, <ProdRules.ATQ: 3>)]
[<__main__.Token object at 0x000001C600AFD710>, (<NonTerminals.T: 3>, <ProdRules.FR: 5>), (<NonTerminals.F: 5>, <ProdRules.I: 10>)]
[<__main__.Token object at 0x000001C600AE7E10>, (<NonTerminals.R: 4>, <ProdRules.EPS: 11>), (<NonTerminals.Q: 2>, <ProdRules.EPS: 11>)]


In [39]:
# print Tree
for pre, fill, node in RenderTree(S):
    print("%s%s" % (pre, node.name))

S


In [41]:
# todo: add code to construct a tree
def parser(tokens_lexemes):
    tokens_lexemes.append(Token(value=4, lexeme="$"))
    per_ip = [tokens_lexemes[0]]
    result = []
    i = 0                                                       #used to keep track of the length of the tokens_lexemes dataframe
    push(stack, NonTerminals.S)                                #push S to stack for the first line
    S = Node(NonTerminals.S.name)
    nodes = []
    while i < len(tokens_lexemes) and len(stack) != 0:   #executes until the stack becomes empty and the tokens_lexmes dataframe reaches the end
        # print(stack)
        x = changeinput(tokens_lexemes[i].name, tokens_lexemes[i].lexeme)       #changes input to the instance of Terminals class; will be used further as column no for parser table
        y = pop(stack)                                                            #pops the first value of the stack; will be used further as row no for parser table
        # tree
        if 
        #
        if (x == Terminals.NONE or y == NonTerminals.NONE):                       #this will raise a Syntax Error which will be caught by Error Handling
            raise SyntaxError("Token: {}\nLexeme: {} \nSyntax Error: Invalid Syntax".format(tokens_lexemes[i].name, tokens_lexemes[i].lexeme))
        elif (y in LIST_TERMINALS):                             #executes if y and x are both equal
            if (x == y):
                result.append(per_ip)
                i+=1
            else:
                raise SyntaxError("Token: {}\nLexeme: {} \nSyntax Error: Invalid Syntax".format(tokens_lexemes[i].name, tokens_lexemes[i].lexeme))
            per_ip = [tokens_lexemes[i]]           #used to keep record of the token and lexeme for the current iteration; will be added to op_df when the input changes
        else:                                                                   #executes when y and x are not equal
            new_value = PREDICTIVE_PARSER_TABLE[y.value][x.value]                       #calculates the Production Rule according to y and x from the parser table
            per_ip.append((y, new_value))                                          #adds the Prod Rule used to the op_ls list
            pushRules(new_value)                                                  #pushes the contents of the Prod Rule to the stack
    result.append(per_ip)
    return result

In [43]:
tree

[[<__main__.Token at 0x1c600afd4a8>,
  (<NonTerminals.S: 0>, <ProdRules.IEE: 0>)],
  (<NonTerminals.E: 1>, <ProdRules.TQ: 2>),
  (<NonTerminals.T: 3>, <ProdRules.FR: 5>),
  (<NonTerminals.F: 5>, <ProdRules.I: 10>)],
  (<NonTerminals.R: 4>, <ProdRules.EPS: 11>),
  (<NonTerminals.Q: 2>, <ProdRules.ATQ: 3>)],
  (<NonTerminals.T: 3>, <ProdRules.FR: 5>),
  (<NonTerminals.F: 5>, <ProdRules.I: 10>)],
  (<NonTerminals.R: 4>, <ProdRules.EPS: 11>),
  (<NonTerminals.Q: 2>, <ProdRules.EPS: 11>)]]

In [59]:
S = Node(tree[0][1][0].name)
add_prod_rules(tree[0][1][1], node_parent=S)
tree_stack = []
for i in tree[1:]:
    rules = i[1:]
    lexeme = i[0]
    for j in rules:
        tree_stack.append(j)
    
print(RenderTree(S))

Node('/S')
├── Node('/S/E')
├── Node('/S/EQUAL')
└── Node('/S/IDENTIFIER')


In [60]:
tree_stack

[(<NonTerminals.E: 1>, <ProdRules.TQ: 2>),
 (<NonTerminals.T: 3>, <ProdRules.FR: 5>),
 (<NonTerminals.F: 5>, <ProdRules.I: 10>),
 (<NonTerminals.R: 4>, <ProdRules.EPS: 11>),
 (<NonTerminals.Q: 2>, <ProdRules.ATQ: 3>),
 (<NonTerminals.T: 3>, <ProdRules.FR: 5>),
 (<NonTerminals.F: 5>, <ProdRules.I: 10>),
 (<NonTerminals.R: 4>, <ProdRules.EPS: 11>),
 (<NonTerminals.Q: 2>, <ProdRules.EPS: 11>)]